In [1]:
### Author: Leonie Küchenhoff
### Date: October 2022
### Purpose of script: Annotate txt files and add info if variant is on pos. or negative strand. Save annotated df in annotation_dir

In [2]:
# package import
import pandas as pd
import os
from config import basedir, annotation_dir

In [3]:
gff_path = '../DNA/variant_calling/inputs/fasta//mm10_refGene.txt'

In [4]:
gff = pd.read_csv(gff_path, delimiter = '\t', skiprows = 1, header = None, 
    names = ['transcript_id','chr','strand','tstart', 'tend', 'cstart' ,'cend', 'n_exons', 'estart', 'eend', 'score', 'name', 'status_start_anno', 'status_end_anno', 'exon_offsets'])

In [5]:
# directory settings
os.chdir(basedir)
annodir = annotation_dir
print('This base directory will be used:\n', basedir)
os.chdir(basedir)

This base directory will be used:
 /g/steinmetz/project/leonie_crispr/03_data/02_rnaseq/snakemake/


In [6]:
# sample names
names = ['028_pbs_R', '029_pbs_R', '032_pbs_R','033_nrch_R', '030_nrch_R', '036_nrch_R', 
            '011_pbs', '012_nrch', '013_nrch', '014_nrch', '279_spry', '321_pbs', '333_pbs', '450_spry', '283_spry']

In [7]:
# paths to input files
paths_som = [f'filtered_tables/HL{i}.specific.txt' for i in names]
paths_germ = [f'filtered_tables/HL{i}.tisoverlap.txt' for i in names]
# paths_annotated file
paths_anno_hc = [f'{annodir}/HL{i}_hc.mm10_multianno.txt' for i in names]
paths_anno_st = [f'{annodir}/HL{i}_st.mm10_multianno.txt' for i in names]
paths_anno_pl = [f'{annodir}/HL{i}_plat.mm10_multianno.txt' for i in names]

In [8]:
# limit columns to read in
use_cols = ['Func.refGene', 'Gene.refGene',
       'GeneDetail.refGene', 'ExonicFunc.refGene', 
       'AAChange.refGene','Otherinfo4', 
       'Otherinfo5', 'Otherinfo7', 'Otherinfo8']

In [9]:
def calc_reads(df, tot_reads_h, tot_reads_l):
    '''
    fuction to add total read counts (alt and ref reads) to df
    '''
    mouse_ad = df[['ad_h','ad_l']]
    allel1 = mouse_ad.applymap(lambda x: int(x.split(',')[0])).to_numpy()
    allel2 = mouse_ad.applymap(lambda x: int(x.split(',')[1])).to_numpy()
    summed = (allel1 + allel2)

    df.loc[:,'reads_h']= summed[:,0]
    df.loc[:,'reads_l']= summed[:,1]
    df.loc[:,'normed_h']= df.loc[:,'reads_h'] / tot_reads_h
    df.loc[:,'normed_l']= df.loc[:,'reads_l'] / tot_reads_l
    return df

In [10]:
# read in data and save in dictionary format for easy accessing throughout the project
unknown_three_dict = {}
known_three_dict = {}
three_dict = {}
for i in zip(paths_som,paths_germ, names, paths_anno_hc, paths_anno_st, paths_anno_pl):
    # read variant files
    som = pd.read_csv(i[0], delimiter = '\t')
    germ = pd.read_csv(i[1], delimiter = '\t')
    # read annnotation files
    anno_dict = {}
    for anno in [i[3],i[4],i[5]]:
        anno_df = pd.read_csv(anno, delimiter = '\t', usecols = use_cols).rename(
            columns={
                'Otherinfo4':'chr', 
                'Otherinfo5':'pos', 
                'Otherinfo7':'ref', 
                'Otherinfo8':'alt'}
            )
        anno_dict[anno] = anno_df
    # concatenate all annotation files so that only one megre is necessary
    anno = pd.concat([anno_dict[anno] for anno in [i[3],i[4],i[5]]]).drop_duplicates()
    # merge variant file with annotation file
    som_merged = pd.merge(som, anno, how = 'left',on = ['chr', 'pos', 'ref', 'alt'])
    # also merge with gff file to add strand information
    som_merged = som_merged.merge(gff[['name', 'strand']].drop_duplicates(), left_on = 'Gene.refGene', right_on = 'name', how = 'left')
    # repeat with variants in heart & liver
    germ_merged = pd.merge(germ, anno, how = 'left',on = ['chr', 'pos', 'ref', 'alt'])
    germ_merged = germ_merged.merge(gff[['name', 'strand']].drop_duplicates(), left_on = 'Gene.refGene', right_on = 'name', how = 'left')

    # read total amount of reads per file (this info will also bee added to txt files)
    path_h = f'bams/readcount/H{i[2]}.out'
    with open(path_h) as f:
        tot_reads_h = int(next(f))
    path_l = f'bams/readcount/L{i[2]}.out'
    with open(path_l) as f:
        tot_reads_l = int(next(f))
    # trim df to columns of interest
    som_merged_f = som_merged[
        (som_merged['Func.refGene'] == 'exonic') |
        (som_merged['Func.refGene'] == 'UTR3') |
        (som_merged['Func.refGene'] == 'UTR5') |
        (som_merged['Func.refGene'] == 'UTR5;UTR3') |
        (som_merged['Func.refGene'] == 'ncRNA_exonic') |
        (som_merged['Func.refGene'] == 'ncRNA_intronic') |
        (som_merged['Func.refGene'] == 'ncRNA_splicing') |
        (som_merged['Func.refGene'] == 'exonic;splicing') 
        ]
    # add total read counts to df
    som_merged_f = calc_reads(som_merged_f, tot_reads_h, tot_reads_l)
    som_merged_f.to_csv(f'filtered_tables/HL{i[2]}.specific.annofilter.txt', sep = '\t')

    germ_merged_f = germ_merged[
        (germ_merged['Func.refGene'] == 'exonic') |
        (germ_merged['Func.refGene'] == 'UTR3') |
        (germ_merged['Func.refGene'] == 'UTR5') |
        (germ_merged['Func.refGene'] == 'UTR5;UTR3') |
        (germ_merged['Func.refGene'] == 'ncRNA_exonic') |
        (germ_merged['Func.refGene'] == 'ncRNA_intronic') |
        (germ_merged['Func.refGene'] == 'ncRNA_splicing') |
        (germ_merged['Func.refGene'] == 'exonic;splicing') 
        ]
    germ_merged_f = calc_reads(germ_merged_f, tot_reads_h, tot_reads_l)
    germ_merged_f.to_csv(f'filtered_tables/HL{i[2]}.tisoverlap.annofilter.txt', sep = '\t')

/tmp/ipykernel_41987/3641543821.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'reads_h']= summed[:,0]
/tmp/ipykernel_41987/3641543821.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'reads_l']= summed[:,1]
/tmp/ipykernel_41987/3641543821.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_